In [ ]:
from votekit.cvr_loaders import load_scottish
from votekit.elections import STV, Borda, Plurality

from functools import partial
from glob import glob
from tqdm.notebook import tqdm
import json
from fairness_metric import sigma_IIA, sigma_UM
import numpy as np

In [2]:
# ELECTION_CLASS = partial(Borda, tiebreak = "first_place")
# election_type = f"plurality"
# ELECTION_CLASS = partial(Plurality, tiebreak = "first_place")
n_approval = 3
election_type = f"{n_approval}-approval"

output_file = f"./scottish_stats/{election_type}_output.json"
stats_file = f"./scottish_stats/{election_type}_stats.json"

In [3]:
scottish_election_stats = {
    str(cands): {
        "n_voters": [],
        "sigma_UM": [],
        "sigma_IIA": [],
    }
    for cands in range(3, 15)
}


all_files = glob("./scot-elex/*/*.csv")
for f in tqdm(all_files):
    n_cands = f.split("/")[-2].split("_")[0]
    approval_vector = [1] * n_approval + [0] * (int(n_cands) - n_approval)
    ELECTION_CLASS = partial(Borda, tiebreak = "first_place", score_vector = approval_vector)
    profile, seats, cand_list, cand_to_party, ward = load_scottish(f)
    scottish_election_stats[n_cands]["n_voters"].append(int(profile.df["Weight"].sum()))
    scottish_election_stats[n_cands]["sigma_UM"].append(float(sigma_UM(profile, ELECTION_CLASS)))
    scottish_election_stats[n_cands]["sigma_IIA"].append(float(sigma_IIA(profile, ELECTION_CLASS)))


  0%|          | 0/1070 [00:00<?, ?it/s]

In [4]:
with open(output_file, 'w') as f:
    json.dump(scottish_election_stats, f, indent=4)

In [5]:
scottish_election_interpreted_values = {
    str(cands): { }
    for cands in range(3, 15)
}

for key, data in scottish_election_stats.items():
    scottish_election_interpreted_values[key]["median_voters"] = int(np.median(data["n_voters"]))
    scottish_election_interpreted_values[key]["max_voters"] = int(np.max(data["n_voters"]))
    scottish_election_interpreted_values[key]["mean_sigma_IIA"] = float(np.mean(data["sigma_IIA"]))
    scottish_election_interpreted_values[key]["variance_sigma_IIA"] = float(np.var(data["sigma_IIA"]))
    scottish_election_interpreted_values[key]["mean_sigma_UM"] = float(np.mean(data["sigma_UM"]))
    scottish_election_interpreted_values[key]["variance_sigma_UM"] = float(np.var(data["sigma_UM"]))
    

In [6]:
with open(stats_file, 'w') as f:
    json.dump(scottish_election_interpreted_values, f, indent=4)